In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

from Generator import Generator

## Parameters

#### Data Generation

In [ ]:
low = 0.1
high = 1.0
rank = True
train_examples = 1000
test_examples = 100

#### Model

In [ ]:
embed_dim = 512
stack = 8

optimizer = keras.optimizers.Nadam()
loss = keras.losses.MSE
metrics = keras.metrics.MeanSquaredError()

#### Training

In [ ]:
batch_size = 128
epochs = 100
validation_split = 0.2
verbose = 2

## Generate Data

In [ ]:
generator = Generator('nutrients.csv') # filename with data

In [ ]:
generator.generate(train_examples) # generates arg number of examples
generator.scale(low,high) # scales ingredients individually between min and max arg values
if rank == True:
    generator.rank()

recipes = generator.recipes # recipes
quant = generator.quant # quantity of ingredients

## Build Model

In [ ]:
embed_dim = 512
stack = 8
i1 = keras.layers.Input(recipes.shape[1:])
f1 = keras.layers.Flatten()(i1)
y1 = keras.layers.Dense(embed_dim)(f1)

# Stack residual blocks (skip connections)
for _ in range(stack):
    y2 = y1
    y1 = keras.layers.Dense(embed_dim,
        activation=keras.activations.relu)(y1)
    y1 = keras.layers.Add()([y1,y2])
    y1 = keras.layers.LayerNormalization()(y1)

o1 = keras.layers.Dense(recipes.shape[1])(y1)
    
model = keras.Model(i1,o1)

In [ ]:
keras.utils.plot_model(model,to_file='recipe_model.png',show_shapes=True,expand_nested=True)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=optimizer,
            loss=loss,
            metrics=metrics)

## Train

In [ ]:
history = model.fit(
    recipes, quant, 
    batch_size=batch_size, 
    epochs=epochs, 
    validation_split = validation_split, 
    verbose=verbose
)

## Testing

In [ ]:
#prediction values
test_gen = Generator('nutrients.csv')
test_gen.generate(test_examples) # generates arg number of examples
test_gen.scale(low,high) # scales ingredients individually between min and max arg values

if rank == True:
    test_gen.rank()

In [ ]:
test_recipes = test_gen.recipes
test_quant = test_gen.quant

In [ ]:
score = model.evaluate(test_recipes, test_quant, verbose=1)

In [ ]:
model.predict(test_recipes[0:1])